In [1]:
import tensorflow as tf
print(tf.__version__)

2.4.1


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
from keras.datasets import imdb
import warnings
warnings.filterwarnings("ignore")
import numpy as np
from sklearn.naive_bayes import BernoulliNB
from sklearn.model_selection import train_test_split
import os
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.sequence import pad_sequences
# from model import VAE
import keras
from tensorflow.python.framework.ops import disable_eager_execution
from keras import objectives, backend as K
import pickle


disable_eager_execution()
max_length = 200


def read_vocab(path):
    output = open(path, 'rb')
    vocab = pickle.load(output)
    output.close()
    return vocab
glove = read_vocab("/content/drive/MyDrive/MLprj/test2000")

def get_latent(data):
  print("inja")
  print(data.shape)
  reconstructed_model = keras.models.load_model("/content/drive/MyDrive/Copy of modell.h5", compile=False)

  print("model ro khoond")
  def custom_loss():
    def vae_loss(x, x_decoded_mean):
          x = K.flatten(x)
          x_decoded_mean = K.flatten(x_decoded_mean)
          xent_loss = max_length * objectives.binary_crossentropy(x, x_decoded_mean)
          return xent_loss 
    return vae_loss

  vae_loss = custom_loss()

  reconstructed_model.compile(optimizer='Adam',
                              loss=custom_loss(), 
                                  metrics=['accuracy'])
  
  input_layer = reconstructed_model.layers[0]
  encoder = reconstructed_model.get_layer("dense_2")
  print("reconstruct")
  # decoder = reconstructed_model.get_layer("decoded_mean")

  get_encoder_output = K.function([input_layer.input],[encoder.output])
  print("aa")
  get_encoder_output = get_encoder_output([data])[0]
  print("bbb")
  # get_decoder_output = K.function([encoder.output],[decoder.output])
  # get_decoder_output = get_decoder_output([data])[0]
  print("SHAPE:",get_encoder_output.shape)
  return get_encoder_output

words_list = []
vectors = []
word_index = imdb.get_word_index()
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
for i in range(3, 2000): 
    word = reverse_word_index.get(i-3, '?')
    vectors.append(glove[word])
    words_list.append(word)

def print_closest_words(vec):
    closest_dist = 10000000
    closest_word = -100
    counter = 0 
    for v in vectors: 
        dist = torch.norm(v - vec)
        if closest_dist >= dist: 
            closest_word = counter 
            closest_dist = dist

        counter += 1
    return words_list[closest_word]


def get_decoded(data):
  # print("Latent SENTENCE", data[0][0:10])

  all = []
  for sentence in data:
    count = 0
    decoded = []
    for word in sentence:

      decoded.append(print_closest_words(word))
      # count += 1
    all.append(decoded)
  
  print("bade decode")
  return np.array(all)

1646592/1641221 [==============================] - 0s 0us/step


In [5]:
import torch
import numpy as np

def save_vocab(vocab, path):
    with open(path, 'w+') as f:     
        for token, index in vocab.stoi.items():
            f.write(f'{index}\t{token}')

def read_vocab(path):
    vocab = dict()
    with open(path, 'r') as f:
        for line in f:
            index, token = line.split('\t')
            vocab[token] = int(index)
    return vocab
    
def get_embedding(dataset):
    # The first time you run this will download a ~823MB file
          # embedding size = 100
    encoded = []
    # print("DATASET[0]:", dataset[0])
    for sentence in dataset:
        encoded.append(np.array([np.array(glove[word]) for word in sentence]))
    # print("ENCODED[0]", encoded[0][0:10])
    return np.array(encoded)
    
# get_embedding()

In [7]:
!pip3 install hickle

     |████████████████████████████████| 51kB 2.5MB/s 


# Load Dataset

In [8]:
import hickle as hkl
def load_dataset():
  (train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words = 2000) 
  print(train_data.shape)
  print(test_data.shape)
  # gan_test_data = test_data[0:2000]
  # gan_test_labels = test_labels[0:2000]
  
  array_hkl = hkl.load('/content/drive/MyDrive/MLprj/data_new2000.hkl')
  X_train = array_hkl['xtest']
  gan_test_labels = array_hkl['ytest']
  word_index = imdb.get_word_index()
  reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
  gan_test_data = []
  print(X_train.shape)
  for i  in  range(len(X_train)):
    decoded_review = [reverse_word_index.get(i - 3, '?') for i in X_train[i]]
    gan_test_data.append(decoded_review)

  niave_total_data = np.concatenate((train_data, test_labels[10000:]), axis = 0)
  naive_total_labels = np.concatenate((train_labels, test_labels[10000:]), axis = 0)

  return gan_test_data, gan_test_labels, niave_total_data, naive_total_labels

gan_test_data, gan_test_labels, niave_total_data, naive_total_labels = load_dataset()

17465344/17464789 [==============================] - 0s 0us/step
(25000,)
(25000,)
(10000, 200)


# Niave Bayes

In [9]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

filename = '/content/drive/MyDrive/MLprj/naive_model.sav'

def preprocess(data):

      truncated_data = []  
      MAX_LENGTH= 200
      for row in data:
          if type(row) == type([1,1]):
              truncated_data.append(row[:MAX_LENGTH])
      data = np.array(truncated_data)
      # print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>", data.shape)
      # print(data[0])


      def vectorize_sequences(sequences, dimension=2000):
          results = np.zeros((len(sequences), dimension))    # Creates an all zero matrix of shape (len(sequences),10K)
          for i,sequence in enumerate(sequences):
              results[i,sequence] = 1   
                          # Sets specific indices of results[i] to 1s
          return results

      # Vectorize training Data
      # print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>", data.shape)
      # print(data[0])
      
      data = vectorize_sequences(data)
      # print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>", data.shape)
      # print(data[0])


      # MAX_LENGTH= 2000
      # data = pad_sequences(data, maxlen=MAX_LENGTH)
      # print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>", data.shape)
      # print(data[0])
      return data


def train_naive():
    X = preprocess(niave_total_data)
    

    y = naive_total_labels
    print(X.shape)
    print(y.shape)
    print("\n")
    kf = KFold(n_splits=10, random_state=None, shuffle = False) 
    # X_train , Y_train, X_test, Y_test = load_data_Naive()
    # X = np.concatenate((X_train, X_test), axis=0)
    # y = np.concatenate((Y_train, Y_test), axis=0)
    # i = 0
    train_accs = []
    test_accs = []
    best_acc = 0
    for train_index, test_index in kf.split(X):
          # print("Train:", train_index, "Validation:",test_index)
          x_train, x_test = X[train_index], X[test_index] 
          y_train, y_test = y[train_index], y[test_index]
          bnb = BernoulliNB(binarize=0.0)
          bnb.fit(x_train, y_train)
          y_pred = bnb.predict(x_train)
          train_acc = accuracy_score(y_train, y_pred)
          print("accuracy:", train_acc)
          train_accs.append(train_acc)
          test_acc = bnb.score(x_test, y_test)
          print("SCORE",test_acc)
          test_accs.append(test_acc)
          if train_acc > best_acc:
            print("THIS IS the max acc", train_acc)
            best_acc = train_acc
            pickle.dump(bnb, open(filename, 'wb'))
          print("*************\n")
    print("Average Train Accuracy:", np.mean(np.array(train_accs)))
    print("Average Test Score:", np.mean(np.array(test_accs)))

train_naive()


def get_naive_model():
    # load the model from disk
    loaded_model = pickle.load(open(filename, 'rb'))
    return loaded_model

get_naive_model()

(25000, 2000)
(40000,)


accuracy: 0.8259555555555556
SCORE 0.806
THIS IS the max acc 0.8259555555555556
*************

accuracy: 0.8250222222222222
SCORE 0.8252
*************

accuracy: 0.8262666666666667
SCORE 0.8256
THIS IS the max acc 0.8262666666666667
*************

accuracy: 0.8261333333333334
SCORE 0.8136
*************

accuracy: 0.8264888888888889
SCORE 0.8224
THIS IS the max acc 0.8264888888888889
*************

accuracy: 0.8264888888888889
SCORE 0.8212
*************

accuracy: 0.8270666666666666
SCORE 0.8064
THIS IS the max acc 0.8270666666666666
*************

accuracy: 0.8252
SCORE 0.824
*************

accuracy: 0.826
SCORE 0.8196
*************

accuracy: 0.8255555555555556
SCORE 0.8204
*************

Average Train Accuracy: 0.8260177777777779
Average Test Score: 0.8184400000000001


BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)

# GAN

In [10]:
import os
from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten
from keras.layers import BatchNormalization, LSTM, Dropout
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.sequence import pad_sequences
from keras import backend as K
from keras.datasets import imdb
import matplotlib.pyplot as plt
import numpy as np
from sklearn.naive_bayes import BernoulliNB
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.python.framework.ops import disable_eager_execution
from sklearn.naive_bayes import BernoulliNB
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")
import hickle as hkl
from sklearn.metrics import accuracy_score


class GAN():

    def __init__(self):
        
        self.vector_size = 200
        self.txt_shape = (self.vector_size,50,)
        optimizer = Adam(0.0002, 0.5)
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
                                   optimizer=optimizer,
                                   metrics=['accuracy'])
        self.batch_size = 10
        self.generator = self.build_generator()
        self.total_loss = 200
        self.array = np.zeros( shape = (self.batch_size,1))
        self.array[0] += self.total_loss

        # z = Input(shape=(self.latent_dim,))
        data = Input(shape=self.txt_shape)
        noise = self.generator(data)
        noise = tf.expand_dims(noise, axis=2)
        noisy_img = data + noise
        # print(noisy_img.shape)
        # noisy_image = K.sum(noise, data)

        self.discriminator.trainable = False
        validity = self.discriminator(noisy_img)
        self.combined = Model(data, validity)
        self.combined.compile(loss= self.get_total_loss, optimizer=optimizer)

        self.founded_examples = []
  
    def get_total_loss(self, fake, validity):
      ep = 0.000001
      t_loss = 3*(K.sum((-1)*(fake * K.log(validity) + (1 - fake) * K.log(1 - validity)))) + self.array[0]

      return t_loss



    def build_generator(self):

        model = Sequential()
        model.add(LSTM(128))
        model.add(Dense(50))
        model.add(LeakyReLU(alpha=0.1))
        model.add(Dropout(0.2))
        model.add(Dense(self.vector_size, activation='tanh'))
        # model.summary()
        data = Input(shape = self.txt_shape)
        noise = model(data)

        return Model(data, noise)

    def build_discriminator(self):
        LSTM_units = 128
        model = Sequential()
        model.add(LSTM(LSTM_units))
        model.add(Dense(100))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(10))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation='sigmoid'))
        # model.summary()
        img = Input(shape = self.txt_shape)
        validity = model(img)
        return Model(img, validity)
        

    def create_model_checkpoint(self, dir, model_name, epoch):
      filepath = dir + '/' + \
                model_name + "-{epoch:%d}.h5"%epoch
      directory = os.path.dirname(filepath)

      try:
          os.stat(directory)
      except:
          os.mkdir(directory)

      checkpointer = ModelCheckpoint(filepath=filepath,
                                    verbose=1,
                                    save_best_only=False)

      return checkpointer

    def save_decoded_original(self,data, path):
        output = open(path, 'wb')
        pickle.dump(data, output)
        output.close()

    def load_decoded_original(self, path):
        output = open(path, 'rb')
        data = pickle.load(output)
        output.close()
        return data  

    def find_good_review(self, data_array, label):
        find = True
        number = 200
        if find: 
            newdata = []
            newlabel = []
            for i in range(len(data_array)):
                sentence_started = False
                count = 0
                for word in data_array[i]:
                    if word != "?" and (not sentence_started):
                          sentence_started = True
                    if sentence_started and word == "?":
                          count += 1
                    if count >6: 
                          break 
                if count <6: 
                    newdata.append(data_array[i])
                    newlabel.append(label[i])
                    number -= 1
                if number <=0: 
                    return  newdata, np.array(newlabel)

            print("Only found: ", 200 -number)
            return  newdata, np.array(newlabel)

        return np.array(data_array)[0:number], np.array(label)[0:number]

    def pre_process(self, X_test):
      new_test = []
      tmp = []
      for sentence in X_test:
        for word in sentence:
          if word == '?':
            tmp.append(0)
          else:
            tmp.append(word_index[word])
        new_test.append(list(tmp))
      x_test = np.array(new_test)
      
      def vectorize_sequences(sequences, dimension=2000):
          results = np.zeros((len(sequences), dimension))    # Creates an all zero matrix of shape (len(sequences),10K)
          for i,sequence in enumerate(sequences):
              results[i,sequence] = 1                        # Sets specific indices of results[i] to 1s
          return results

      MAX_LENGTH= 200
      # Vectorize testing Data

      truncated_data = []  

      for row in x_test:
          if type(row) == type([1,1]):
              truncated_data.append(row[:MAX_LENGTH])
      x_test = np.array(truncated_data)

      X_test = vectorize_sequences(x_test)
      # X_test = pad_sequences(X_test, maxlen=MAX_LENGTH)
     
      return X_test

    def truncate_noisy_data(self, noisy, original):
      truncated_sentence_original= ""
      index = 0
      indx_list = []
      for i in range(len(original)):
          for j in range(len(original[i])):
              if j >0 :
                  if  original[i][j-1] == original[i][j] == '?' and original[i][j+1]!='?':
                    truncated_sentence_original = original[i][j+1:]
                    index  = j+1
                    indx_list.append(index)
                    break
          

          part1 = ['?' for i in range(index)] 
          part2 = [item for item in noisy[i][index:]]
          noisy[i] = part1 + part2

  
          # print("TRUNCATED_ORG", truncated_sentence_original)

      return noisy, indx_list


    def naive(self, bnb, X_test, Y_test):
  
      X_test = self.pre_process(X_test)
      print("SCORE",bnb.score(X_test, Y_test))
      
      def CrossEntropy(pred, y):

              ep = 0.000001
              pos = np.sum(y * -np.log(1 - pred + ep))
              neg = np.sum((1-y) * -np.log(pred + ep))
              # print(type(pos))
              neg + pos
            
              return neg + pos


      def CrossEntropy_Naive(pred, y):
        ep = 0.000001
        pos = np.sum(y * -np.log(pred + ep))
        neg = np.sum((1-y) * -np.log(1-pred + ep))
       
        neg + pos
        return neg + pos

      predicts = bnb.predict(X_test)
      loss = CrossEntropy(predicts, Y_test)
      loss_naive = CrossEntropy_Naive(predicts, Y_test)

      return loss, loss_naive, predicts

    def train(self, EPOCH, batch_size, alpha=0.5):
        data = []
        naive_model = get_naive_model()
        # data_array, label = self.load_data()
        data_array , label = gan_test_data, gan_test_labels
        data, label = self.find_good_review(data_array, label)
        niave_loss_1,loss_naive_1, pred_init = self.naive(naive_model, data, label)
        print("***")

        new_data = []
        for i in range(len(data)):
          if pred_init[i] ==1 and label[i] ==1:
            new_data.append(data[i])
          if pred_init[i] ==0 and label[i] ==1:
            print("wrong prediction")

        data = np.array(new_data)

        encoded = get_embedding(data)
        # all_original = get_decoded(encoded)
        # print("saving original")
        # self.save_decoded_original(all_original, "/content/drive/MyDrive/MLprj/encoded_original.hkl")
        # print("reading original")
        all_original = self.load_decoded_original("/content/drive/MyDrive/MLprj/encoded_original.hkl")
        all_original = np.array(all_original)
        print("done with original")

        print(niave_loss_1,loss_naive_1)
         
        print("SHAPE ENCODED")
        print(encoded.shape)
        X_train = encoded
        
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(EPOCH):
          print('avvale epoch')
          print("epoch:",epoch)
          idx = np.random.randint(0, X_train.shape[0], batch_size)


          imgs = X_train[idx]
          print("SELECTED INDEX:", idx)
          # original =  get_decoded(imgs)
          original = all_original[idx] 
          loss_1,loss_naive_1, pred1 = self.naive(naive_model, original, np.array([1 for i in range((original.shape)[0])]))
          print("loss_naive_not_noisy",loss_naive_1 )
          print("loss_modified_not_noisy", loss_1)

          checkpointer = self.create_model_checkpoint('/content/drive/MyDrive/MLprj/', 'GAN', epoch)
          g_loss = self.combined.fit(imgs,valid,epochs=1, batch_size=batch_size, callbacks=[checkpointer])
          gen_imgs = self.generator.predict(imgs)

          gen_imgs = np.expand_dims(gen_imgs, axis=2)

          noisy_image = 6*gen_imgs + imgs

          decoded_noise = get_decoded(noisy_image)
          print(decoded_noise[0])
          print("test")
          # decoded_noise, index_list = self.truncate_noisy_data(decoded_noise,  original)

          # print("NOISE", decoded_noise)
          # print("NOISY OUT NUM", noisy_image)

          # print("truncated!!!!!!")

          # for i in range(len(index_list)): 
          #       noisy_image[i] = list(imgs[i][:index_list[i]]) + list(noisy_image[i][index_list[i]:])
          # noisy_image = np.array(noisy_image)

          d_loss_real = self.discriminator.fit(imgs, valid, epochs=1, batch_size=batch_size)
          d_loss_fake = self.discriminator.fit(noisy_image, fake, epochs=1, batch_size=batch_size, steps_per_epoch=10)

          # d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
          # noisy_image = tf.squeeze(noisy_image, axis=2)


          naive_input = decoded_noise
          loss_2, loss_naive_2, pred2 = self.naive(naive_model, naive_input, np.array([1 for i in range((naive_input.shape)[0])]))

      


          for i in range(len(pred1)): 
            if pred1[i] != pred2[i]: 
                  print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")
                  print("foundddd!!!!!",  pred1[i],  pred2[i]) 
                  print("Review:", original[i])
                  print("NOISY Review",naive_input[i])
                  self.founded_examples.append({"original": original[i], "noisy": naive_input[i], "index": i, "allindex": idx, "pred": [pred1[i],  pred2[i]]})
                  print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")
          print("\n\n")
          print("loss_naive_noisy",loss_naive_2 )
          print("loss_modified_noisy", loss_2)

          # self.total_loss = np.float32(niave_loss_2)
          self.total_loss = np.float32(loss_2)

          # self.array = np.zeros( shape = (self.batch_size,1))
          self.array[0] = self.total_loss
          print("akhare epoch")

        #     print("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100 * d_loss[1], g_loss))
        # if epoch % sample_interval == 0:
        #     self.sample_images(epoch)



gan = GAN()
gan.train(EPOCH=30, batch_size=gan.batch_size)

SCORE 0.515
***
trying to decode
done with original
1422.9974904703686 1340.1044211225862
SHAPE ENCODED
(103, 200, 50)
avvale epoch
epoch: 0
SELECTED INDEX: [23 27 47 10 32 15 34 77 45 98]
SCORE 1.0
loss_naive_not_noisy -9.999994999180669e-06
loss_modified_not_noisy 138.15510557964274
Train on 10 samples
10/10 [==============================] - ETA: 0s - loss: 221.6497
Epoch 00001: saving model to /content/drive/MyDrive/MLprj/GAN-1.h5
10/10 [==============================] - 2s 164ms/sample - loss: 221.6497
bade decode
['?' '?' 'you' '?' '?' '?' '?' 'you' 'you' '?' '?' '?' '?' '?' '?' '?' '?'
 '?' 'you' '?' '?' '?' '?' '?' '?' '?' '?' '?' '?' '?' '?' 'you'
 'remember' '?' '?' '?' '?' 'you' '?' '?' '?' 'you' '?' '?' '?' '?' '?'
 'america' '?' 'you' '?' '?' '?' 'you' '?' 'you' '?' '?' 'black' '?' '?'
 '?' '?' '?' '?' '?' '?' '?' '?' '?' '?' '?' '?' '?' '?' '?' 'you' '?' '?'
 '?' 'you' '?' 'you' '?' '?' '?' 'you' 'you' '?' 'you' '?' '?' '?' '?' '?'
 'you' '?' 'you' 'you' '?' '?' '?' 'you'

KeyboardInterrupt: ignored

# `Reconstruct`

In [ ]:
reconstructed_model = keras.models.load_model("/content/drive/MyDrive/MLprj/GAN_NEW.h5", compile=False)
# input  = np.random.rand(10, 300, 1)
gan = GAN()
data_array, label = gan.load_data()
data = []
for i in range(len(data_array)):
  if label[i] ==1:
    data.append(data_array[i])

data = np.array(data)[10:12]
encoded = get_embedding(data)
input = encoded
print("DATA",data)

total_loss = 200
array = np.zeros( shape = (10,1))
array[0] += total_loss

def custom_loss():
  def vae_loss(fake, validity):
        ep = 0.000001
        t_loss = (-1)*(fake * K.log(validity) + (1 - fake) * K.log(1 - validity)) + array
        print(t_loss)
        print("****")
        print(K.mean(t_loss))
        print(type(K.mean(t_loss)))
        print("finished")
        return K.mean(t_loss)
  return vae_loss

vae_loss = custom_loss()
reconstructed_model.compile(optimizer='Adam',
                            loss=custom_loss())

print(reconstructed_model.summary())
input_layer = reconstructed_model.layers[0]
g_layer = reconstructed_model.get_layer("tf_op_layer_add_18")
out = K.function([input_layer.input],[g_layer.output])
out = out([input])[0]
# print(out)
naive_input = get_decoded(out)
print("DEOCED,", naive_input)


# a = reconstructed_model.predict(input)

# print(a)



  

In [ ]:
print(out.shape)
# # a = tf.math.reduce_sum([1,1,1])

# a = np.array([1,2,3])
# c = np.array([4,5,3])
# b = K.constant(a)
# c = K.constant(c)
# print(1-b)
# l=np.log(b)
# # ep = 0.00000110]
# print(l)
# # b + ep
m = [9,10]

arr = np.array([[9, 9, 9, 9, 9, 9], [9,9,9,9,9,10]])
occurrences = np.count_nonzero(arr == 9)
print(occurrences)
